In [10]:
import numpy as np
import cantera as ct
from NetGEN.netsmoke import *

In [11]:
# Setup simple CRN: two PSR in series with recirculation

# First of all, set path to kinetic mechanisms
kinfile     = "/Users/matteosavarese/Desktop/Dottorato/Kinetics/GRI3.0/gri30.CKI"
thermofile  = "/Users/matteosavarese/Desktop/Dottorato/Kinetics/GRI3.0/thermo.dat"
canteramech = 'gri30.yaml' 

# Path to NetSMOKEpp.sh
netsmoke_path = "/Users/matteosavarese/Desktop/Dottorato/Github/NetSMOKEpp/SeReNetSMOKEpp-master/projects/Linux"

In [12]:
# Create a reference Cantera object and the various streams
gas = ct.Solution(canteramech)
gas.TP = 300.0, 101325.0        

# Set equivalence ratio of the inlet mixture
fuel = 'CH4:1'
oxi  = 'O2:0.21, N2:0.79'
gas.set_equivalence_ratio(1.0, fuel=fuel, oxidizer=oxi) # by default basis is mol

In [13]:
# Create the reactor object
M0 = 1.0    # reference 1.0 kg/s
r0 = Reactor('PSR', isothermal=False, tau=1.0, Mf=M0, P=gas.P, 
                 InletMixture=gas, InitialStatus=gas, sp_threshold=1e-5,
                 CanteraMech=canteramech,
                 isinput=True)

# Select recirculation ratio
kv = 1.00

# Create flame reactor
M1 = M0*(1+kv)  # total inlet mass into M1 (kg/s)
r1 = Reactor('PSR', isothermal=False, volume=0.343, Mf=M1, P=gas.P, 
                 InletMixture=gas, InitialStatus=gas, sp_threshold=1e-5,
                 CanteraMech=canteramech, Uq=1500, Aq=1.0, Tenv=300.0, 
                 isoutput=True) # the volume is ULB furnace volume

### Create connection matrix ###
# element i-j is the mass going from reactor i to j
# inlet outlet are specified as inputs to reactors and should not be considered here
rconns = np.array([[0.0, M1],
                   [M1-M0, 0.0]])

# Create reactors list
rlist = [r0, r1]

# Create reactor network object
rn = ReactorNetwork(rlist, rconns, kinfile, thermofile)
rn.WriteNetworkInput(FolderName="SimpleCRN")

In [14]:
# Run simulation
rn.RunSimulation(netsmoke_path, verbose=False)

# Extract outputs 
Mlist = rn.ExtractOutputs()
print("\n\n========= State of mixture in combustion reactor ======== \n\n")
print(Mlist[0].report())

print("\n\n========= State of mixture in post-flame reactor ======== \n\n")
print(Mlist[1].report())



========= State of mixture in combustion reactor ======== 



  gri30:

       temperature   1802.3 K
          pressure   1.0132e+05 Pa
           density   0.18649 kg/m^3
  mean mol. weight   27.58 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -1.0021e+06       -2.7637e+07  J
   internal energy       -1.5454e+06       -4.2622e+07  J
           entropy            9506.1        2.6218e+05  J/K
    Gibbs function       -1.8135e+07       -5.0016e+08  J
 heat capacity c_p            1471.2             40575  J/K
 heat capacity c_v            1169.7             32261  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2        8.7429e-05         0.0011961           -25.881
                 H        1.9741e-06        5.4016e-05           -11.487
       